In [1]:
from ch2.data import data
d = data('-v', '5')
stats = d.statistic_names('Fatigue', 'Fitness', 'Heart Rate', 'HR Zone', 'HR Impulse')
print(stats)
#data = d.statistic_journals('Fatigue', 'Fitness', start='2017-06-01', finish='2017-07-01')
data = d.statistic_journals('Fatigue', 'Fitness', 'Heart Rate', 'HR Zone', 'HR Impulse', 'HR Impulse (duration)', 
                            start='2017-06-06 15:45', finish='2017-06-06 15:55')
print(data.describe())

    INFO: Using database at /home/andrew/.ch2/database.sqll
   DEBUG: SELECT statistic_name.name, statistic_journal.time, coalesce(statistic_journal_float.value, statistic_journal_integer.value, statistic_journal_text.value) AS coalesce_1 
FROM statistic_journal JOIN statistic_name ON statistic_name.id = statistic_journal.statistic_name_id LEFT OUTER JOIN statistic_journal_float ON statistic_journal.id = statistic_journal_float.id LEFT OUTER JOIN statistic_journal_integer ON statistic_journal.id = statistic_journal_integer.id LEFT OUTER JOIN statistic_journal_text ON statistic_journal.id = statistic_journal_text.id 
WHERE statistic_name.id IN (:id_1, :id_2, :id_3, :id_4, :id_5, :id_6, :id_7) AND statistic_journal.time >= :time_1 AND statistic_journal.time <= :time_2 ORDER BY statistic_journal.time
   DEBUG: Loaded 44 distinct times


         name description units summary                owner  \
0     Fatigue        None  None    None    ImpulseStatistics   
1     Fitness        None  None    None    ImpulseStatistics   
2  HR Impulse        None  None    None  HeartRateStatistics   
3     HR Zone        None  None    None  HeartRateStatistics   
4     HR Zone        None  None    None  HeartRateStatistics   
5  Heart Rate        None   bpm    None     ActivityImporter   

             constraint   count  
0  ActivityGroup "Bike"  441791  
1  ActivityGroup "Bike"  441791  
2  ActivityGroup "Bike"  441004  
3  ActivityGroup "Bike"  442979  
4   ActivityGroup "Run"       0  
5  ActivityGroup "Bike"  514396  
       Heart Rate    HR Zone  HR Impulse  HR Impulse (duration)       Fitness  \
count   44.000000  44.000000   44.000000              44.000000     44.000000   
mean   123.227273   2.765289    2.945823              13.681818  36606.009900   
std      7.181577   0.460465    3.088841              13.061167     36

In [9]:
import datetime as dt
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Dark2
from bokeh.models import LinearAxis, Range1d

output_notebook()

f = figure(plot_width=500, x_axis_type='datetime', plot_height=400, title='Typical Impulse Data')

f.xaxis.axis_label = 'Time'
f.extra_y_ranges = {'Response': Range1d(start=35400, end=36300),
                    'HR Impulse': Range1d(start=0, end=0.3)}
f.y_range = Range1d(start=0, end=3.5)
f.yaxis.axis_label = 'HR Zone'
f.add_layout(LinearAxis(y_range_name='Response', axis_label='Response (Fitness, Fatigue)'), 'right')
f.add_layout(LinearAxis(y_range_name='HR Impulse', axis_label='HR Impulse / sec'), 'left')

duration = pd.Series([np.nan if pd.isnull(x) else dt.timedelta(seconds=x) for x in data['HR Impulse (duration)']])

f.vbar(x=data.index-0.5*duration, top=data['HR Impulse']/data['HR Impulse (duration)'], 
       width=duration, color=Dark2[5][4], fill_color='white', legend='HR Impulse', y_range_name='HR Impulse')
f.line(x=data.index, y=data['Fatigue'], color=Dark2[5][1], line_width=2, legend='Fatigue', y_range_name='Response')
# scale fitness to prduce a nicer plot - remember that teh scaling factor is arbitrary anyway...
f.line(x=data.index, y=data['Fitness']*0.98, color=Dark2[5][0], line_width=2, legend='Fitness', y_range_name='Response')
f.line(x=data.index, y=data['HR Zone'], color=Dark2[5][3], line_width=2, legend='HR Zone')

f.legend.location = 'bottom_right'    
show(f)

Loading BokehJS ...